## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.reset_default_graph()

### Collect Data

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [7]:
data = data.drop(labels=['date', 'symbol'], axis=1)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [9]:
data_subset = data.head(1000)
data_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float64
close     1000 non-null float64
low       1000 non-null float64
high      1000 non-null float64
volume    1000 non-null float64
dtypes: float64(5)
memory usage: 39.1 KB


In [10]:
features = data_subset.drop(labels='volume', axis=1)

In [11]:
target = data_subset['volume']/1000000

In [12]:
target.head(2)

0    2.1636
1    2.3864
Name: volume, dtype: float64

In [13]:
data_subset.head(2)

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0


### Divide the data into train and test sets

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [16]:
X_train = X_train.values
X_test = X_test.values

In [17]:
y_train = y_train.values
y_test = y_test.values

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [38]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

In [39]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define a function to calculate prediction

In [40]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
return y

<tf.Tensor 'output:0' shape=(?, 1) dtype=float32>

3.Loss (Cost) Function [Mean square error]

In [41]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [45]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record math ops on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])# pylint: disable=W0212
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss w.r.t Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [46]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [47]:
for i in range(100):
    
    w, b = train(X_train, y_train, W, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)).numpy())

SystemError: <built-in function TFE_Py_TapeWatch> returned a result with an error set

### Get the shapes and values of W and b

In [ ]:
w.shape

In [ ]:
b.shape

### Model Prediction on 1st Examples in Test Dataset

In [ ]:
train(X_test, y_test, w, b)
print('Current Loss on iteration', i, loss(y_test, prediction(X_test, w, b)).numpy())

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

In [48]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Load the given Iris data using pandas (Iris.csv)

In [50]:
df = pd.read_csv("Iris.csv")

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [53]:
iris = pd.concat([df,pd.get_dummies(df['Species'], prefix='Species')],axis=1)

In [55]:
iris.drop(['Species'],axis=1, inplace=True)

In [58]:
iris.drop(['Id'],axis=1, inplace=True)

In [59]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 7 columns):
SepalLengthCm              150 non-null float64
SepalWidthCm               150 non-null float64
PetalLengthCm              150 non-null float64
PetalWidthCm               150 non-null float64
Species_Iris-setosa        150 non-null uint8
Species_Iris-versicolor    150 non-null uint8
Species_Iris-virginica     150 non-null uint8
dtypes: float64(4), uint8(3)
memory usage: 5.2 KB


### Splitting the data into feature set and target set

In [60]:
X = iris.drop(labels=['Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica'], axis=1)
Y = iris.drop(labels=['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm'], axis=1)

In [61]:
X.head(1)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2


In [62]:
Y.head(1)

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0


### Divide the dataset into Training and test (70:30)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [64]:
model = Sequential()
# First hidden layer with 12 neurons and input as 4 features  
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
# Output layer
model.add(Dense(3, activation='softmax'))
# Compile model as a graph
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Model Training 

In [65]:
model.fit(X_train, y_train, epochs=100, batch_size=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
105/105 [==============================] - 1s 5ms/step - loss: 1.2059 - acc: 0.3333
Epoch 2/100
105/105 [==============================] - 0s 1ms/step - loss: 1.1006 - acc: 0.3429
Epoch 3/100
105/105 [==============================] - 0s 914us/step - loss: 1.0951 - acc: 0.3524
Epoch 4/100
105/105 [==============================] - 0s 952us/step - loss: 1.0707 - acc: 0.3429
Epoch 5/100
105/105 [==============================] - 0s 952us/step - loss: 1.0207 - acc: 0.4857
Epoch 6/100
105/105 [==============================] - 0s 952us/step - loss: 0.9255 - acc: 0.6190
Epoch 7/100
105/105 [==============================] - 0s 1ms/step - loss: 0.7609 - acc: 0.7048
Epoch 8/100
105/105 [==============================] - 0s 1ms/step - loss: 0.6702 - acc: 0.7048
Epoch 9/100
105/105 [==============================] - 0s 1ms/step - loss: 0.6188 - acc: 0.7238
Epoch 10/100
105/105 [==============================] - 0s 1ms/step - loss: 0.57

### Model Prediction

In [66]:
scores = model.evaluate(X_test, y_test)
scores[1]*100

45/45 [==============================] - 0s 2ms/step


97.77777777777777

### Save the Model

In [67]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?